<span style="color:red;font-weight:bold">Jayant Solanki</span>,
<span style="color:red;font-weight:bold">Anant Gupta</span>
<hr/>
## <span style="float:left">Lab 3</span>
### <span style="float:right">DATA ANALYTICS PIPELINE USING APACHE SPARK</span>
#### <span style="float:right">NY Times article collection and creation of the csv file</span>
<hr/>

In [12]:
#Background: The nytimes api doesn't returns full article body, just the snippet of it each returned
# so, I have to write the code to fetch the weburl for each article, then perfrom further requests to those url and 
#fetch the story body of each article
import requests # for performing html request to nytimes API
from datetime import datetime
from time import sleep
import lxml.html as html # for scrapping the content of the article URL of nytimes
import json
import os
import re
from nltk.corpus import stopwords
import string

In [26]:
stop_word_list = stopwords.words('english')
#  to quickly test if a word is not a stop word, use a set:
stop_word_set = set(stop_word_list)
# Each API keys can provide 10000 response hits
# topic = ["baseball", "politics","market","fashion"]#topic to be looked for
# topic = ["soccer", "election","economy","movies"]#topic to be looked for
# topic = ["sport", "trump","finance","music"]#topic to be looked for
# topic = ["hockey", "north korea","business","entertainment"]#topic to be looked for
# topic = ["basketball", "south china sea","money","dance"]#topic to be looked for
topic = ["basketball", "south china sea","money","celebrity"]#topic to be looked for

apikeyIndex=0;
apikey = ["ddfe36f13f354e348d9cea28e2b27001", "2c94bb3581cf469fb33321a9e3bbac38","f4918bd3047241889c282af42bd2128a","ddfe36f13f354e348d9cea28e2b27001"]#add more apikeys here
fl = "snippet,web_url"#selective attributes of json response
pageNo = "0"#initial page is 0, articles fetched using api are grouped in 10 per page starting 0 and upto page 100
dateRange = ["20180501", "20180502","20180503", "20180504","20180505", "20180506","20180507", "20180508"]
category = ["sports", "politics","business","entertainment"]
# dateRange = ["20180321", "20180322", "20180323", "20180324", "20180325", "20180326", "20180327", "20180328"]# can be changed t any period

In [27]:
#this function parse the json response from nytimes and create new dictionary using two attributes only
def parse_articles(articles):
    '''
    This function takes in a response to the NYT api and parses
    the articles into a list of dictionaries
    '''
    news = []
    fetch = articles['response']['docs']
    for i in range(0,len(fetch)):
        dic = {}
#         print(fetch[i])
        dic['web_url'] = fetch[i]['web_url']
        if fetch[i]['snippet'] is not None:
            dic['snippet'] = fetch[i]['snippet']
#         dic['url'] = i['web_url']
        news.append(dic)
    return(news)

In [28]:
# this function perfrom request to nytimes api using the paramters passed and returns the parseed responsed to the caller function
def get_articles(topic, begin_date, end_date, fl, apikey, apikeyIndex):
    all_articles = []#stores all articles for a particular day
    page = 0
    while(page<100):
        sleep(1)
#     for page in range(0,100): #NYT limits pages to first 100 pages starting page 0, each page has 10 articles max
        try:
            
            url = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q="+topic+"&begin_date="+begin_date+"&end_date="+end_date+"&fl="+fl+"&page="+str(page)+"&api-key="+apikey[apikeyIndex]
            print(url)
            requestArticles = requests.get(url)
            data = requestArticles.json()
            if len(data["response"]["docs"])>0:
                all_articles.append(parse_articles(data))
#                 print(data)
            else:# checks if further pages have no articles to show, if yes then break the loop and return the fetched articles
                print(parse_articles(data))
                break
        except:
            if(data['message']=='API rate limit exceeded'):
                print("You called the api way to fast, Dude, trying again")
                apikeyIndex= apikeyIndex+1
            else:
                print(data)
#             page = page - 1
            sleep(1)
            continue#try again
        print("Page: "+str(page))
#         break
        page=page+1
    return(all_articles)

In [29]:
with open("textcorpus/articles.csv", 'a') as outfile:#for storing full articles in the csv
    for labels in range(3,4):
        print(apikey[apikeyIndex])
        processArticles = []
#         if not os.path.exists("textcorpus"):
#             os.makedirs("textcorpus")
#         if not os.path.exists("textcorpus/"+category[index]):
#             os.makedirs("textcorpus/"+category[index])

        for i in range(0,len(dateRange)-1):
            datetimeobject = datetime.strptime(dateRange[i],'%Y%m%d')
            beginDate = datetimeobject.strftime('%m-%d-%Y')
            datetimeobject = datetime.strptime(dateRange[i+1],'%Y%m%d')
            endDate = datetimeobject.strftime('%m-%d-%Y')
            print("Fetching articles for Data period: " + beginDate + " - "+ endDate)
            processArticles = get_articles(topic[labels], dateRange[i], dateRange[i+1],fl, apikey, apikeyIndex)
            if(len(processArticles)>0):
        #         try:
        #             dataToWrite = processArticles
        # #             print(dataToWrite)
        #         except:
        #             print(len(processArticles))
        #             print(processArticles)
        #             print(processArticles[0])
        #             break
    #                 print("processing snippets")
    #                 count=0
    #                 for item in processArticles:
    #                     page=0
    #                     for articles in item:
    #                         count=count+1
    #                         page=page+1
    #                         articlesCount="page"+str(page)+"articles"+str(count)
    #                         with open("textcorpus/"+category[index]+"/"+topic[index]+articlesCount+dateRange[i]+".txt", 'w') as outfile:#used for storing snippets

    #         #                     print(articles)
    #                             outfile.write(articles["snippet"])
    #                             outfile.write("\n")
                    print("processing full articles")
                    count=0
                    for item in processArticles:
                        page=0
                        for articles in item:
                            count=count+1
                            page=page+1
                            articlesCount="page"+str(page)+"articles"+str(count)
                            fullpage = requests.get(articles["web_url"])
                            htmlbody = html.fromstring(requests.get(articles["web_url"]).content)
                            output = "".join(htmlbody.xpath('//p[contains(@class,"story-body-text")]//text()'))#scrapper
                            output = output.join(htmlbody.xpath('//div[contains(@class,"StoryBodyCompanionColumn")]//text()'))#scrapper
        #                     if(len(output)<100):
        #                         break
        #                     print(len(output))
        #                     print(output+"\n\n")
        #                     output = output.decode('utf8').encode('latin1').decode('utf8')
                            output = str(output.encode("ascii", "ignore"))#since the body has lots of escape characters, I have to convert
            # utf-8 response into ascii using ignore flag to bypass those escape characters
                            # do filtering of useless characters here
                            output = output.strip(',')
                            output=output.strip('"')
                            output=output.strip('.')
                            output=output.strip('â€œ')
                            output=output.strip('$')
                            output=output.strip('?')
                            output=output.strip(')')
                            output=output.strip('(')
                            output=output.strip(' ')
                            output   = re.sub(r'\$\w*', '', output)
                            output   = re.sub(r'http?:.*$', '', output)
                            output   = re.sub(r'https?:.*$', '', output)
                            output   = re.sub(r'pic?.*\/\w*', '', output)
                            output   = re.sub(r'[' + string.punctuation + ']+', ' ', output)  # Remove puncutations like 's
                            output=str.replace(output,'\'s','')
                            output=str.replace(output,'\\','')
                            output=str.replace(output,'s\'','')
                            output=str.replace(output,',','')
                            output = output.upper()
                            if(len(output[2:-1])<5):
                                          continue
                            #appending with userfull columns
                            output=dateRange[i]+","+category[labels]+","+topic[labels]+","+str(page)+","+output[2:-1]
                            outfile.write(output)
                            outfile.write("\n")
            else:
                print("Insufficient data for date: "+beginDate+" to save")
        #     break
        # print(processArticles[0][0:-1])
print("Fetching Completed")

ddfe36f13f354e348d9cea28e2b27001
Fetching articles for Data period: 05-01-2018 - 05-02-2018
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=celebrity&begin_date=20180501&end_date=20180502&fl=snippet,web_url&page=0&api-key=ddfe36f13f354e348d9cea28e2b27001
You called the api way to fast, Dude, trying again
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=celebrity&begin_date=20180501&end_date=20180502&fl=snippet,web_url&page=0&api-key=2c94bb3581cf469fb33321a9e3bbac38
Page: 0
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=celebrity&begin_date=20180501&end_date=20180502&fl=snippet,web_url&page=1&api-key=2c94bb3581cf469fb33321a9e3bbac38
[]
processing full articles
Fetching articles for Data period: 05-02-2018 - 05-03-2018
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=celebrity&begin_date=20180502&end_date=20180503&fl=snippet,web_url&page=0&api-key=ddfe36f13f354e348d9cea28e2b27001
You called the api way to fast, Dude, trying again
http://api.nytimes.